In [300]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from pykml import parser
import re
import pgeocode

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [301]:
with open('Worldwide_Board_Game_Cafe_List.kml','r') as f:
    doc = parser.parse(f).getroot()

In [302]:
url = 'https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_Europe'
response = requests.get(url)
response

<Response [200]>

In [303]:
soup = BeautifulSoup(response.content, 'lxml')
table = soup.find_all('table', attrs={'class':'wikitable'})[1]
eu_countries = [td.a.text for td in table.find_all('td')[2::7]]

In [304]:
eu_countries.remove('United Kingdom')
eu_countries.extend(['England', 'Scotland', 'Wales', 'Northern Ireland'])
eu_countries.sort()

In [305]:
eu_countries

['Albania',
 'Andorra',
 'Armenia',
 'Austria',
 'Azerbaijan',
 'Belarus',
 'Belgium',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'England',
 'Estonia',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Hungary',
 'Iceland',
 'Ireland',
 'Italy',
 'Kazakhstan',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Moldova',
 'Monaco',
 'Montenegro',
 'Netherlands',
 'North Macedonia',
 'Northern Ireland',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'San Marino',
 'Scotland',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'Vatican City',
 'Wales']

In [ ]:
cafe_list = []

In [ ]:
for e in doc.Document.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
    cafe_dict = dict()
    if e.ExtendedData.Data[5].value in eu_countries:
        cafe_dict = { 
            'Name': e.name.text,
            'City': e.ExtendedData.Data[2].value.text,
            'PostalCode': e.ExtendedData.Data[4].value.text,
            'Country': e.ExtendedData.Data[5].value.text
                    }
        cafe_list.append(cafe_dict)

In [308]:
for item in cafe_list:
    if item['Country'] == 'Czech Republic':
        item['Country'] = 'Czechia'

In [309]:
cafe_df = pd.DataFrame(cafe_list)
cafe_df.head(10)

,Name,City,PostalCode,Country
0,Brot & Spiele,Graz,8020,Austria
1,Brot und Spiele,Vienna,1080,Austria
2,Café Benno,Vienna,1080,Austria
3,Café Sperlhof,Vienna,1020,Austria
4,SpielBar,Vienna,1080,Austria
5,The Playground (Hoofdkerk),Antwerp,2000,Belgium
6,The Playground (Station),Antwerp,2018,Belgium
7,Outpost Antwerpen,Antwerpen,2000,Belgium
8,The Playground,Antwerpen,2000,Belgium
9,La Luck Brussels,Brussels,1050,Belgium


In [310]:
gn_url = 'http://www.geonames.org/countries/'
gn_response = requests.get(gn_url)
gn_soup = BeautifulSoup(gn_response.content, 'lxml')

In [311]:
gn_table = gn_soup.find_all('tr')[2:]
code_list = []
for tr in gn_table:
    code_dict = dict()
    td = tr.find_all('td')
    if td[-1].text in ['EU', 'AS']: # We include AS because GeoNames includes Turkey within Asia
        code_dict = {
            'Country': td[4].text,
            'Code': td[0].text
                    }
        code_list.append(code_dict)


In [312]:
code_df = pd.DataFrame(code_list)

In [313]:
uk_countries = [{'Country': 'England', 'Code': 'GB'}, 
                {'Country': 'Scotland', 'Code': 'GB'}, 
                {'Country': 'Wales', 'Code': 'GB'}, 
                {'Country': 'Northern Ireland', 'Code': 'GB'}]

In [314]:
code_df = pd.concat([code_df, pd.DataFrame(uk_countries)])
code_df.set_index('Country', inplace=True)
code_df.drop('United Kingdom', axis=0, inplace=True)
code_df.sort_index()

,Code
Country,
Afghanistan,AF
Albania,AL
Andorra,AD
Armenia,AM
Austria,AT
Azerbaijan,AZ
Bahrain,BH
Bangladesh,BD
Belarus,BY


In [315]:
cafe_df = cafe_df.merge(code_df, how='left', on='Country')
cafe_df.head(10)

,Name,City,PostalCode,Country,Code
0,Brot & Spiele,Graz,8020,Austria,AT
1,Brot und Spiele,Vienna,1080,Austria,AT
2,Café Benno,Vienna,1080,Austria,AT
3,Café Sperlhof,Vienna,1020,Austria,AT
4,SpielBar,Vienna,1080,Austria,AT
5,The Playground (Hoofdkerk),Antwerp,2000,Belgium,BE
6,The Playground (Station),Antwerp,2018,Belgium,BE
7,Outpost Antwerpen,Antwerpen,2000,Belgium,BE
8,The Playground,Antwerpen,2000,Belgium,BE
9,La Luck Brussels,Brussels,1050,Belgium,BE


In [316]:
def get_coords(row):
    geo = pgeocode.Nominatim(row[-1])
    coords = geo.query_postal_code(row[2])
    return [coords.latitude, coords.longitude]

In [317]:
coord_list = []

In [318]:
for row in cafe_df.values:
    coord_dict = dict()
    try:
        ll = get_coords(row)
    except:
        ll = [np.nan, np.nan]
    coord_dict = {
        'Latitude': ll[0], 
        'Longitude': ll[1]         
                 }
    coord_list.append(coord_dict)

In [319]:
coord_df = pd.DataFrame(coord_list)
coord_df.head(10)

,Latitude,Longitude
0,47.0232,15.5337
1,48.2167,16.3500
2,48.2167,16.3500
3,48.2167,16.4000
4,48.2167,16.3500
5,51.2199,4.4035
6,51.2199,4.4035
7,51.2199,4.4035
8,51.2199,4.4035
9,50.8333,4.3667


Let's check to see if there are any cafés that could not provide coordinates.

In [321]:
null_idx = coord_df.index[coord_df['Latitude'].isnull()].tolist()
null_cafes = cafe_df.iloc[null_idx]
null_cafes

,Name,City,PostalCode,Country,Code
23,3 Trolls,Sofia,1505 Oborishte,Bulgaria,BG
42,The Games Table,Norwich,NR311JF,England,GB
73,Dice Saloon,Brighton,BN14GY,England,GB
124,Game of trolls,Moulins,3000,France,FR
192,Playhouse,Athens,106 81,Greece,GR
193,Κάισσα Cafe,Athens,11527,Greece,GR
194,Playce,Athina,115 24,Greece,GR
195,Playhouse,Ioannina,453 33,Greece,GR
196,Playhouse,Kavala,652 01,Greece,GR
197,Playhouse,Larisa,412 22,Greece,GR


Well, that's a number of missing data pieces. On a hunch, let's first count the number of cafés per country in both `cafe_df` and `null_cafes`.

In [323]:
cafe_df.groupby('Country').count()

,Name,City,PostalCode,Code
Country,,,,
Austria,5,5,5,5
Belgium,15,15,15,15
Bulgaria,4,4,4,4
Czechia,3,3,3,3
Denmark,11,11,11,11
England,78,78,78,78
Finland,2,2,2,2
France,57,57,57,57
Germany,17,17,17,17


In [324]:
null_cafes.groupby('Country').count()

,Name,City,PostalCode,Code
Country,,,,
Bulgaria,1,1,1,1
England,2,2,2,2
France,1,1,1,1
Greece,11,11,11,11
Ireland,1,1,1,1
Italy,1,1,1,1
Moldova,1,1,1,1
Netherlands,8,8,8,8
Serbia,2,2,2,2


By comparing these numbers, we can see that every café in Greece, Moldova and Serbia are missing, which suggests that `pgeocode` does not contain information for these countries (and a check of the [github page](https://github.com/symerio/pgeocode) for `pgeocode` confirms this). This will require utilising a different source for the information.
Otherwise, the postal codes for the remaining entries of `null_cafes` must contain errors and need to be manually changed.

In [329]:
cafe_df.drop(23, axis=0, inplace=True) # A search reveals that 3 Trolls in Bulgaria is permanently closed
cafe_df.loc[42, 'PostalCode'] = 'NR2 1EL'
cafe_df.loc[73, 'PostalCode'] = 'BN1 4JF'
cafe_df.loc[124, 'PostalCode'] = '03000'
cafe_df.loc[211, 'PostalCode'] = 'P75 AE30'
cafe_df.drop(220, axis=0, inplace=True) # This is in fact related to an event called Counters in Pontypridd, Wales, not Italy
cafe_df.loc[223, 'PostalCode'] = 'MD-2012'
cafe_df.loc[230, 'PostalCode'] = '9712 NP'
cafe_df.loc[231, 'PostalCode'] = '2011 LE'
cafe_df.loc[286, 'PostalCode'] = '411 19'
cafe_df.loc[291, 'PostalCode'] = '06490'

So, what is next to sort out is a) making sure we can obtain the coordinates for Greece, Moldova and Serbia (and Netherlands); and b) remove any duplicate entries within `cafe_df`.

Starting with the duplicates...

* Antwerpen is the Dutch name for Antwerp, and so 'The Playground' in Antwerpen is duplicate of one of the two 'The Playground's already listed within the city.


In addition, we need to change the city names to match---e.g. the above issue with Antwerpen vs. Antwerp, and Bruxelles and Brussels---in order to later group things by city.

In [331]:
cafe_df.loc[7, 'City'] = 'Antwerp'
cafe_df.drop(8, axis=0, inplace=True)
cafe_df.loc[12, 'City'] = 'Brussels'
cafe_df.drop(33, axis=0, inplace=True)
cafe_df.drop(44, axis=0, inplace=True)
cafe_df.drop(50, axis=0, inplace=True)
cafe_df.drop(74, axis=0, inplace=True)
cafe_df.drop(79, axis=0, inplace=True)
cafe_df.loc[104, 'Name'] = 'Nerdy Coffee Co.'
cafe_df.loc[194, 'City'] = 'Athens'
cafe_df.drop(206, axis=0, inplace=True)
cafe_df.loc[207, 'Name'] = 'Pub Game Up!'
cafe_df.drop(212, axis=0, inplace=True)
cafe_df.drop(264, axis=0, inplace=True)
cafe_df.drop(272, axis=0, inplace=True)

,Name,City,PostalCode,Country,Code
0,Brot & Spiele,Graz,8020,Austria,AT
1,Brot und Spiele,Vienna,1080,Austria,AT
2,Café Benno,Vienna,1080,Austria,AT
3,Café Sperlhof,Vienna,1020,Austria,AT
4,SpielBar,Vienna,1080,Austria,AT
5,The Playground (Hoofdkerk),Antwerp,2000,Belgium,BE
6,The Playground (Station),Antwerp,2018,Belgium,BE
7,Outpost Antwerpen,Antwerp,2000,Belgium,BE
9,La Luck Brussels,Brussels,1050,Belgium,BE
10,La Table Food & Games,Brussels,1000,Belgium,BE


In checking `cafe_df` we can see that cafés are not yet group by cities, so let's do that and then reset the indices.

In [340]:
cafe_df.sort_values(['Country', 'City'], inplace=True)
cafe_df.reset_index(drop=True, inplace=True)
cafe_df

,Name,City,PostalCode,Country,Code
0,Brot & Spiele,Graz,8020,Austria,AT
1,Brot und Spiele,Vienna,1080,Austria,AT
2,Café Benno,Vienna,1080,Austria,AT
3,Café Sperlhof,Vienna,1020,Austria,AT
4,SpielBar,Vienna,1080,Austria,AT
5,The Playground (Hoofdkerk),Antwerp,2000,Belgium,BE
6,The Playground (Station),Antwerp,2018,Belgium,BE
7,Outpost Antwerpen,Antwerp,2000,Belgium,BE
8,La Luck Brussels,Brussels,1050,Belgium,BE
9,La Table Food & Games,Brussels,1000,Belgium,BE
